In [1]:
%pip install --upgrade elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from elasticsearch import Elasticsearch
import numpy as np

# Connect to Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

from deepface import DeepFace
import os
import pandas as pd

img_dir = "/var/wd_smit/internal_repos/backups/images"
models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "Dlib", "ArcFace"]
embeddings = []
face_coords = []
names = []

df = pd.DataFrame({"embedding": [], "facial_area": []})

for img_name in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_name)
    embedding = DeepFace.represent(img_path, model_name=models[0])
    embeddings.append(embedding[0]["embedding"])
    face_coords.append(embedding[0]["facial_area"])
    names.append(img_name)

    dftemp = pd.DataFrame(embedding)
    df = pd.concat([df, dftemp], ignore_index=True)

df["name"] = names

print(face_coords)
print(df.info)

for i in range(len(df['embedding'])):
    df['embedding'][i] = np.array(df['embedding'][i])

# Index the DataFrame into Elasticsearch
index_name = 'embedding'
for _, row in df.iterrows():
    document = row.to_dict()
    es.index(index=index_name, id = 1, body=document)




2023-09-24 04:42:20.182249: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-24 04:42:20.184564: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 04:42:20.232377: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 04:42:20.233111: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 04:42:21.057956: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

[{'x': 97, 'y': 26, 'w': 82, 'h': 82}, {'x': 102, 'y': 30, 'w': 79, 'h': 79}, {'x': 150, 'y': 176, 'w': 519, 'h': 519}, {'x': 38, 'y': 56, 'w': 96, 'h': 96}]
<bound method DataFrame.info of                                            embedding  \
0  [0.009040644392371178, -0.01007477194070816, 0...   
1  [0.010348817333579063, 0.0034686659928411245, ...   
2  [0.009841795079410076, 0.020586512982845306, 0...   
3  [0.00940479151904583, -0.0032933431211858988, ...   

                                facial_area  \
0      {'x': 97, 'y': 26, 'w': 82, 'h': 82}   
1     {'x': 102, 'y': 30, 'w': 79, 'h': 79}   
2  {'x': 150, 'y': 176, 'w': 519, 'h': 519}   
3      {'x': 38, 'y': 56, 'w': 96, 'h': 96}   

                                     name  
0                            Untitled.jpg  
1                           Untitled2.jpg  
2  Elon_Musk_Colorado_2022_(cropped2).jpg  
3                           Untitled1.jpg  >


In [3]:
testpath = "/var/wd_smit/internal_repos/backups/images/Untitled2.jpg"
testembed = embedding = DeepFace.represent(testpath, model_name=models[0])

query_vector = testembed[0]['embedding']

# Search for similar vectors
search_body = {
    'query': {
        'script_score': {
            'query': {'match_all': {}},
            'script': {
                'source': "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                'params': {'query_vector': query_vector}
            }
        }
    }
}
search_results = es.search(index=index_name, body=search_body)
print(search_results)

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'runtime error')

In [ ]:
testpath = "/var/wd_smit/internal_repos/backups/images/Untitled2.jpg"
testembed = embedding = DeepFace.represent(testpath, model_name=models[0])

In [ ]:
testembed

In [ ]:
query_vector = testembed[0]['embedding']
query_vector

In [ ]:
# len(query_vector)

In [ ]:
hosts = [{"host": "_local_", "port": "9200"}]

In [ ]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Define the knn search parameters
knn = {
  "field": "embedding",
  "query_vector": query_vector,
  "k": 2,
  "num_candidates": 100
}

# Define the source fields to return
source = ["name", "facial_area"]

# Perform the knn search on the index "articles"
res = es.search(index="embedding", knn=knn, source=source)

res